In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

<h1>Load hasil baca berita</h1>

In [2]:
def loadPickle(name):
    file = open(name, 'rb')
    data = pickle.load(file)
    file.close()
    
    return data

In [3]:
def listOfDictToDataFrame(listOfDict):
    dataFrame = pd.DataFrame(listOfDict)
    
    x = dataFrame.iloc[:, :-1].values
    y = dataFrame.iloc[:, 4].values
    
    return dataFrame, x, y

<h1>Bangun Label encoder</h1>

In [4]:
forX = preprocessing.LabelEncoder()
forY = preprocessing.LabelEncoder()
forNewX = preprocessing.LabelEncoder()

In [5]:
def labelEncoderX(x):
    temp = x[0] #[ 0 'sss' 'sss' 'sss']
    x = np.delete(x,0) #['sss' 'sss' 'sss']
    
    forX.fit(x)
    x = forX.transform(x) #[2 2 1]
    x = np.append(x,temp) #[2 2 1 0]
    
    return x

In [6]:
def labelEncoderNewX(x):
    temp = x[0] #[ 0 'sss' 'sss' 'sss']
    x = np.delete(x,0) #['sss' 'sss' 'sss']
    
    forNewX.fit(x)
    x = forNewX.transform(x) #[2 2 1]
    x = np.append(x,temp) #[2 2 1 0]
    
    return x

<h1>Baca Data Test</h1>

In [7]:
def extFeature(words):
    listOfDictFeature = []
    i = 0
    for i in range(len(words)): #iterasi dari awal tiap kata
        dictFeature = {}
        
        dictFeature = inputToDict(words, i)
        
        listOfDictFeature.append(dictFeature)
        
    return listOfDictFeature

def inputToDict(words, i):
    dictFeature = {}
    
    dictFeature['index'] = i    
    dictFeature['first_upper_case'] = isFirstUpper(words[i])
    dictFeature['prev_word'] = isEdge(i, words)
    dictFeature['word'] = words[i]
    
    return dictFeature

def isFirstUpper(word):
    if word[0].isupper():
        return 'y'
    else:
        return 'n'
    
def isEdge(i, words):
    if i == 0:
        return 'null'
    else:
        return words[i-1]

In [8]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

<h1>Proses ==============</h1>

In [9]:
data = loadPickle("training.pickle")
dataFrame, x, y = listOfDictToDataFrame(data)

In [10]:
forx = OneHotEncoder(handle_unknown='ignore')
forx.fit(x)
newx = forx.transform(x).toarray()
print(len(newx),len(newx[0]))

1198 1969


In [11]:
#forY biar string jadi skala
forY = preprocessing.LabelEncoder()
forY.fit(y)
y = forY.transform(y)
print(len(y))

1198


<h3>Bangun classifier KNN</h3>

In [12]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(newx, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [13]:
testWord = readPerWord("test.txt")
listOfDictFeature = extFeature(testWord)
df = pd.DataFrame(listOfDictFeature)

#mengubah data frame ke list
newdf = df.iloc[:, :].values

In [14]:
fortest = OneHotEncoder(handle_unknown='ignore')
fortest.fit(df)
a = fortest.transform(df).toarray()

<h3>Prediksi</h3>

In [15]:
y_pred = classifier.predict(a)

ValueError: query data dimension must match training data dimension

In [ ]:
print(y_pred)

In [ ]:
y_pred_invers = list(forY.inverse_transform(y_pred))
print(y_pred_invers)

<h3>Hasil</h3>

In [ ]:
person = []
location = []
organization = []
for i in range(len(df)):
    if y_pred_invers[i] == 'person':
        person.append(df['word'][i])
    elif y_pred_invers[i] == 'location':
        location.append(df['word'][i])
    elif y_pred_invers[i] == 'organization':
        organization.append(df['word'][i])

In [ ]:
print(person)
print(location)
print(organization)